# 🧠 Grafana Dashboard Summarizer
Simulate reading a Grafana dashboard JSON and summarize its panels using GPT or plain logic.

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import json
import pandas as pd

In [2]:


with open("mock_grafana_dashboard.json", "r") as f:
    data = json.load(f)

dashboard = data["dashboard"]
panels = dashboard["panels"]
print(f"Dashboard Title: {dashboard['title']}")
print(f"Total Panels: {len(panels)}\n")
for p in panels:
    print(f"- {p['title']} ({p['type']})")


Dashboard Title: Mock System Metrics Dashboard
Total Panels: 4

- CPU Usage (%) (stat)
- Memory Usage (gauge)
- Disk Read Errors (stat)
- GPU Usage (%) (gauge)


In [3]:
# Extracting panel data

panel_data = []
for p in panels:
    thresholds = p.get("fieldConfig", {}).get("defaults", {}).get("thresholds", {}).get("steps", [])
    panel_data.append({
        "Title": p["title"],
        "Type": p["type"],
        "Unit": p.get("fieldConfig", {}).get("defaults", {}).get("unit", "N/A"),
        "Thresholds": thresholds
    })

df = pd.DataFrame(panel_data)
df


,Title,Type,Unit,Thresholds
0,CPU Usage (%),stat,percent,"[{'color': 'green', 'value': None}, {'color': ..."
1,Memory Usage,gauge,bytes,"[{'color': 'green', 'value': None}, {'color': ..."
2,Disk Read Errors,stat,short,"[{'color': 'green', 'value': None}, {'color': ..."
3,GPU Usage (%),gauge,percent,"[{'color': 'green', 'value': None}, {'color': ..."


In [4]:

summary_prompt = f"""
You are a helpful assistant summarizing a system monitoring dashboard.

Dashboard: {dashboard['title']}
Panels:
"""
for idx, row in df.iterrows():
    summary_prompt += f"- {row['Title']} [{row['Type']}] - Unit: {row['Unit']}, Thresholds: {row['Thresholds']}\n"

print(summary_prompt)



You are a helpful assistant summarizing a system monitoring dashboard.

Dashboard: Mock System Metrics Dashboard
Panels:
- CPU Usage (%) [stat] - Unit: percent, Thresholds: [{'color': 'green', 'value': None}, {'color': 'orange', 'value': 70}, {'color': 'red', 'value': 90}]
- Memory Usage [gauge] - Unit: bytes, Thresholds: [{'color': 'green', 'value': None}, {'color': 'orange', 'value': 12000000000.0}, {'color': 'red', 'value': 14000000000.0}]
- Disk Read Errors [stat] - Unit: short, Thresholds: [{'color': 'green', 'value': None}, {'color': 'orange', 'value': 5}, {'color': 'red', 'value': 10}]
- GPU Usage (%) [gauge] - Unit: percent, Thresholds: [{'color': 'green', 'value': None}, {'color': 'orange', 'value': 75}, {'color': 'red', 'value': 90}]



In [5]:


load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [6]:
openai = OpenAI()

In [7]:
def summarize():
    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a Grafana dashboard summarizer."},
        {"role": "user", "content": summary_prompt}
    ]
)
    return response.choices[0].message.content

In [8]:

summary = summarize()
display(Markdown(summary))

### Mock System Metrics Dashboard Summary

**1. CPU Usage (%)**
   - **Type:** Stat
   - **Unit:** Percent
   - **Current Value:** (Value not provided)
   - **Thresholds:**
     - Green: Normal operation
     - Orange: Warning at 70%
     - Red: Critical at 90%

**2. Memory Usage**
   - **Type:** Gauge
   - **Unit:** Bytes
   - **Current Value:** (Value not provided)
   - **Thresholds:**
     - Green: Normal operation
     - Orange: Warning at 12 GB (12,000,000,000 bytes)
     - Red: Critical at 14 GB (14,000,000,000 bytes)

**3. Disk Read Errors**
   - **Type:** Stat
   - **Unit:** Short (number of errors)
   - **Current Value:** (Value not provided)
   - **Thresholds:**
     - Green: Normal operation
     - Orange: Warning at 5 errors
     - Red: Critical at 10 errors

**4. GPU Usage (%)**
   - **Type:** Gauge
   - **Unit:** Percent
   - **Current Value:** (Value not provided)
   - **Thresholds:**
     - Green: Normal operation
     - Orange: Warning at 75%
     - Red: Critical at 90%

### Conclusion
This dashboard provides crucial insights into system performance, monitoring CPU and GPU usage percentages, memory usage in bytes, and the count of disk read errors, helping to maintain optimal system operation and alerting for potential issues.